In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install adabelief_pytorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
import os
import re
import numpy as np
from tqdm import tqdm
from glob import glob
import json
import requests
import tensorflow as tf
from transformers import BertModel, TFBertModel, TFRobertaModel, RobertaTokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel, AutoModelForSequenceClassification
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import *
from sklearn.model_selection import train_test_split,KFold,StratifiedKFold
from adabelief_pytorch import AdaBelief
from transformers.optimization import get_cosine_schedule_with_warmup
from tqdm import tqdm, tqdm_notebook
import shutil
import gc
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import joblib

import warnings
warnings.filterwarnings(action='ignore')

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
path = '/content/drive/MyDrive/에이블스쿨/실습파일/2023.04.17_미니프로젝트 6차_실습자료/'

train_df = joblib.load(path + 'train_data.pkl')
test_df = joblib.load(path + 'test_data.pkl')
clean_train_questions = joblib.load(path + 'clean_train_Q.pkl')
clean_test_questions = joblib.load(path + 'clean_test_Q.pkl')

In [6]:
train_df['intent'] = train_df['intent'] - 1
test_df['intent'] = test_df['intent'] - 1

In [84]:
#random seed 고정
tf.random.set_seed(1234)
np.random.seed(1234)
BATCH_SIZE = 32
NUM_EPOCHS = 40

L_RATE = 1e-5
MAX_LEN = 30
max_grad_norm=1
log_interval=20
NUM_CORES = os.cpu_count()
device = torch.device("cuda:0")

In [85]:
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large", cache_dir='bert_ckpt', do_lower_case=False)

In [86]:
class TrainDataset(Dataset):
    def __init__(self, df):
        self.df_data = df
    def __getitem__(self, index):
        # get the sentence from the dataframe
        sentence = self.df_data.loc[index, 'Q']
        encoded_dict = tokenizer(
          text = sentence,
          add_special_tokens = True, 
          max_length = MAX_LEN,
          pad_to_max_length = True,
          truncation=True,           # Pad & truncate all sentences.
          return_tensors="pt")

        padded_token_list = encoded_dict['input_ids'][0]
        token_type_id = encoded_dict['token_type_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        target = torch.tensor(self.df_data.loc[index, "intent"])
        sample = (padded_token_list, token_type_id , att_mask, target)
        return sample
    def __len__(self):
        return len(self.df_data)

In [87]:
class TestDataset(Dataset):
    def __init__(self, df):
        self.df_data = df
    def __getitem__(self, index):
        # get the sentence from the dataframe
        sentence = self.df_data.loc[index, 'Q']
        encoded_dict = tokenizer(
          text = sentence,
          add_special_tokens = True, 
          max_length = MAX_LEN,
          pad_to_max_length = True,
          truncation=True,           # Pad & truncate all sentences.
          return_tensors="pt")

        padded_token_list = encoded_dict['input_ids'][0]
        token_type_id = encoded_dict['token_type_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        sample = (padded_token_list, token_type_id , att_mask)
        return sample
    def __len__(self):
        return len(self.df_data)

In [88]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [89]:
#AUto
train_data = TrainDataset(train_df)

test_data = TestDataset(test_df)

train_dataloader = torch.utils.data.DataLoader(train_data,
                                        batch_size=BATCH_SIZE,
                                        shuffle=True,
                                      num_workers=NUM_CORES)
test_dataloader = torch.utils.data.DataLoader(test_data,
                                        batch_size=BATCH_SIZE,
                                        shuffle=False,
                                      num_workers=NUM_CORES)
#두가지 모델
#"klue/roberta-large"
#"monologg/koelectra-base-v3-discriminator"
model = AutoModelForSequenceClassification.from_pretrained("klue/roberta-large",num_labels=53)


####미세조정
n=0
for name, child in model.named_children():
    if n==0:
      h=0
      for param in child.parameters():
        if h<=328: #이부분 숫자 조절로 fine-tuning => Roberta229: h=229
          param.requires_grad = False
        h+=1
    n+=1
#####
    # print(param)
model.to(device)
optimizer = AdaBelief(model.parameters(), lr=1e-5, eps=1e-16, betas=(0.9,0.999), weight_decouple = True, rectify = False)

warmup_ratio = 0.1
t_total = len(train_dataloader) * NUM_EPOCHS
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

for e in range(NUM_EPOCHS):
    train_acc = 0.0
    test_acc = 0.0
    best_acc =0.0
    model.train()
    torch.set_grad_enabled(True)
    for batch_id, (input_id,token_type_id,attention_mask,label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        input_id = input_id.long().to(device)
        token_type_id = token_type_id.long().to(device)
        attention_mask = attention_mask.long().to(device)
        label = label.to(device)
        outputs = model(input_ids=input_id, token_type_ids=token_type_id, attention_mask=attention_mask, labels=label)
        loss = outputs[0]
        out = outputs[1]
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

preds = [] 
model.eval()
torch.set_grad_enabled(False)
for batch_id, (input_id,token_type_id,attention_mask) in enumerate(tqdm_notebook(test_dataloader)):
    input_id = input_id.long().to(device)
    token_type_id = token_type_id.long().to(device)
    attention_mask = attention_mask.long().to(device)
    outputs = model(input_ids=input_id, token_type_ids=token_type_id, attention_mask=attention_mask)
    out = outputs[0]
    for inp in out:
      preds.append(inp.detach().cpu().numpy())
Preds = np.array(preds)

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'cla

Please check your arguments if you have upgraded adabelief-pytorch from version 0.0.5.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  ---------
adabelief-pytorch=0.0.5  1e-08  False              False
>=0.1.0 (Current 0.2.0)  1e-16  True               True
SGD better than Adam (e.g. CNN for Image Classification)    Adam better than SGD (e.g. Transformer, GAN)
----------------------------------------------------------  ----------------------------------------------
Recommended eps = 1e-8                                      Recommended eps = 1e-16
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer
You can disable the log message by setting "print_change_log = False", though it is recommended to keep as a reminder.

Weight decoupling enabled in AdaBelief


  0%|          | 0/38 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 4.089357376098633 train acc 0.0
epoch 1 batch id 21 loss 3.9761247634887695 train acc 0.01488095238095238
epoch 1 train acc 0.015625


  0%|          | 0/38 [00:00<?, ?it/s]

...

  0%|          | 0/38 [00:00<?, ?it/s]

epoch 39 batch id 1 loss 0.050370100885629654 train acc 1.0
epoch 39 batch id 21 loss 0.02630315162241459 train acc 0.9985119047619048
epoch 39 train acc 0.9991776315789473


  0%|          | 0/38 [00:00<?, ?it/s]

epoch 40 batch id 1 loss 0.03743357211351395 train acc 1.0
epoch 40 batch id 21 loss 0.03716328367590904 train acc 0.9985119047619048
epoch 40 train acc 0.9991776315789473


  0%|          | 0/4 [00:00<?, ?it/s]

In [90]:
y_pred = Preds.argmax(axis=1)

accuracy_score(test_df['intent'], y_pred)

0.9245283018867925

In [91]:
test_text = '에이블스쿨에 지원하고 싶습니다.'
df = pd.DataFrame([test_text], columns=['Q'])
temp = TestDataset(df)

In [92]:
temp.__getitem__(0)

(tensor([    0,  6154,  2603,  2255,  3028,  2170,  3691, 19521,  1335,  2219,
          3606,    18,     2,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1]),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0]),
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0]))

In [93]:
input_id

tensor([[    0,  4215,  2052,  4366,  1415,  2075,  2182,     2,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1],
        [    0,  5263,  2200,   882,  2069,  1295,  1513,  2075,  2182,     2,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1],
        [    0,  4819,  2103,  2044, 16570,  1513,  2075,  2182,     2,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1],
        [    0,  5142,  3634,  2052, 15770,  2689,  2075,  2182,     2,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1],
        [    0,  3664,  2065,  3691,  5007,  623

In [94]:
tf.expand_dims(input_id.cpu(), axis=0)

<tf.Tensor: shape=(1, 10, 30), dtype=int64, numpy=
array([[[    0,  4215,  2052,  4366,  1415,  2075,  2182,     2,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1],
        [    0,  5263,  2200,   882,  2069,  1295,  1513,  2075,  2182,
             2,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1],
        [    0,  4819,  2103,  2044, 16570,  1513,  2075,  2182,     2,
             1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1],
        [    0,  5142,  3634,  2052, 15770,  2689,  2075,  2182,     2,
             1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,

In [95]:
model.eval()
torch.set_grad_enabled(False)

input_id, token_type_id, attention_mask = temp.__getitem__(0)
input_id = torch.unsqueeze(input_id, 0).long().to(device)
token_type_id = torch.unsqueeze(token_type_id, 0).long().to(device)
attention_mask = torch.unsqueeze(attention_mask, 0).long().to(device)
outputs = model(input_ids=input_id, token_type_ids=token_type_id, attention_mask=attention_mask)
out = outputs[0].detach().cpu().numpy()
out

array([[-0.9242188 ,  1.0527705 ,  0.4423048 , -1.1692629 , -1.492679  ,
         0.9190653 , -0.7380262 , -1.0462441 , -0.59269893, -1.2439088 ,
        -0.9070605 , -0.43042427, -0.7778186 ,  0.12261087, -1.6011982 ,
        -0.11280892,  0.53757584, -0.31467015,  0.43060088,  1.027757  ,
        -0.96574235, -1.2857579 , -0.31905445, -1.3317108 , -0.32740495,
         0.2475657 ,  1.4690833 , -1.4652127 ,  1.8761928 ,  0.17704198,
         4.738522  , -0.4727335 ,  1.7807429 , -0.13860677,  0.42016414,
        -0.45127782,  1.149915  , -0.75003487, -0.9116654 , -0.41731182,
        -0.98348916, -1.7114147 ,  0.6452323 , -0.45351616,  0.6193878 ,
         0.549043  , -0.48898235,  0.06617378, -0.5887444 ,  1.3157808 ,
         2.750411  , -0.31567922,  1.3536314 ]], dtype=float32)

In [96]:
out.argmax(axis=1)

array([30])

In [97]:
temp = train_df[train_df['intent'] == out.argmax(axis=1)[0]]
answer_list = temp['A'].unique()
answer_prob = temp['A'].value_counts(normalize=True).values
np.random.choice(answer_list, 1, p=answer_prob)

array(['KT 에이블스쿨은 정규 4년제 대학 졸업자 및 졸업예정자 중 만 34세 이하 미취업자를 대상으로 하는 교육입니다.\n단, 모집시점에 만 35세여도 해당연도 1월 1일 이후 생일자는 지원이 가능합니다.\n또한 전공의 종류와는 무관 합니다.'],
      dtype=object)

In [98]:
def get_answer(question):
    df = pd.DataFrame([question], columns=['Q'])
    df = TestDataset(df)
    
    model.eval()
    torch.set_grad_enabled(False)

    input_id, token_type_id, attention_mask = df.__getitem__(0)
    input_id = torch.unsqueeze(input_id, 0).long().to(device)
    token_type_id = torch.unsqueeze(token_type_id, 0).long().to(device)
    attention_mask = torch.unsqueeze(attention_mask, 0).long().to(device)
    outputs = model(input_ids=input_id, token_type_ids=token_type_id, attention_mask=attention_mask)
    out = outputs[0].detach().cpu().numpy()
    intent_pred = out.argmax(axis=1)[0]

    temp = train_df[train_df['intent'] == intent_pred]
    answer_list = temp['A'].unique()
    answer_prob = temp['A'].value_counts(normalize=True).values
    return np.random.choice(answer_list, 1, p=answer_prob)[0], intent_pred

In [99]:
question = '오늘 날씨가 어때요?'
ans, intent = get_answer(question)
print("=" * 120)
print("질문 :", question)
print("=" * 120)
print("대답 :", ans)
print("=" * 120)
print("범주 :", intent + 1)
print("=" * 120)

질문 : 오늘 날씨가 어때요?
대답 : 감기 조심 하세요.
범주 : 3


In [100]:
question = '오늘 저녁 뭐먹을까?'
ans, intent = get_answer(question)
print("=" * 120)
print("질문 :", question)
print("=" * 120)
print("대답 :", ans)
print("=" * 120)
print("범주 :", intent + 1)
print("=" * 120)

질문 : 오늘 저녁 뭐먹을까?
대답 : 맛있게 드세요.
범주 : 20


In [101]:
question = '너는 뭐먹고 싶어?'
ans, intent = get_answer(question)
print("=" * 120)
print("질문 :", question)
print("=" * 120)
print("대답 :", ans)
print("=" * 120)
print("범주 :", intent + 1)
print("=" * 120)

질문 : 너는 뭐먹고 싶어?
대답 : 냉장고 파먹기 해보세요.
범주 : 11


In [102]:
question = '휴가는 없나요?'
ans, intent = get_answer(question)
print("=" * 120)
print("질문 :", question)
print("=" * 120)
print("대답 :", ans)
print("=" * 120)
print("범주 :", intent + 1)
print("=" * 120)

질문 : 휴가는 없나요?
대답 : 단위기간(훈련시작일로부터 1개월) 내 80% 이상 출석해야 훈련수강 유지가 가능합니다. 
예를 들어, 훈련시작일로부터 1개월을 기준으로 1개월간의 훈련일수가 20일이라고 가정할 경우 16일 이상 출석하셔야 훈련수강을 계속 유지할 수 있습니다. 
결석 기준은 1. 당일 소정훈련시간의 50퍼센트 미만을 수강한 경우, 2. 지각, 조퇴 3회 누적 시, 결석 처리 됩니다. 
훈련 수준 유지를 위해 100% 출석을 권고 드립니다. 

K-Digital Training (K-DT) 규정상 월 1회 휴가 사용이 가능합니다.
범주 : 41


In [103]:
question = '보강은 없나요?'
ans, intent = get_answer(question)
print("=" * 120)
print("질문 :", question)
print("=" * 120)
print("대답 :", ans)
print("=" * 120)
print("범주 :", intent + 1)
print("=" * 120)

질문 : 보강은 없나요?
대답 : KT 에이블스쿨은 미래인재육성 + 청년 IT/SW/DX 분야 일자리 창출을 위해서 KT 와 정부가 함께 하는 프로그램입니다. 에이블러 여러분들의 취업기회를 위해 Job Searching 세션, 취업컨설팅, 잡페어 등 여러 프로그램들을 운영하고 있습니다.
KT 직접채용은 그 일환 중 하나이며, KT에 합격한 1기 수료생의 경우 해당지역에 배치되었습니다. KT 외에 KT 그룹사, 주요 협력사, AI원팀 기업, 스타트업 등의 채용은 지역 구분 없이 진행됩니다. 채용 상황에 따라 변경될 수 있어 교육 수료 후 채용 연계 시 상세 안내 드리겠습니다. KT 그룹, AI 원팀 기업/스타트업 100 등 다양한 기업에서 채용 연계할 예정으로 기업마다 인재상 및 반영 요소가 다르기 때문에 중점 요소에 대해 안내드리기 어려운 점 양해 부탁드립니다. 채용에 대한 부분은 교육을 참여하시면 추후 관련 부서를 통해 안내드릴 예정입니다.
범주 : 50


In [104]:
question = '수료를 완료하면 수료증을 주나요?'
ans, intent = get_answer(question)
print("=" * 120)
print("질문 :", question)
print("=" * 120)
print("대답 :", ans)
print("=" * 120)
print("범주 :", intent + 1)
print("=" * 120)

질문 : 수료를 완료하면 수료증을 주나요?
대답 : KT 에이블스쿨은 고용노동부 K-Digital Training 사업으로 운영됨에 따라 총 훈련일수의 80% 이상을 출석한 경우 수료대상이 됩니다.
반대로, 총 훈련일수의 20%를 초과하여 결석한 경우 제적대상이 됩니다.
범주 : 42


In [105]:
question = '채용 연계 가능한 협력사는 어디인가요?'
ans, intent = get_answer(question)
print("=" * 120)
print("질문 :", question)
print("=" * 120)
print("대답 :", ans)
print("=" * 120)
print("범주 :", intent + 1)
print("=" * 120)

질문 : 채용 연계 가능한 협력사는 어디인가요?
대답 : KT 에이블스쿨은 미래인재육성 + 청년 IT/SW/DX 분야 일자리 창출을 위해서 KT 와 정부가 함께 하는 프로그램입니다.  
에이블러 여러분들의 취업기회를 위해 Job Searching 세션, 취업컨설팅, 잡페어 등 여러 프로그램들을 운영하고 있습니다.

KT 직접채용은 그 일환 중 하나이며, KT에 합격한 1기 수료생의 경우 해당지역에 배치되었습니다.  
KT 외에 KT 그룹사, 주요 협력사, AI원팀 기업, 스타트업 등의 채용은 지역 구분 없이 진행됩니다. 
채용 상황에 따라 변경될 수 있어 교육 수료 후 채용 연계 시 상세 안내 드리겠습니다. 

KT 그룹, AI 원팀 기업/스타트업 100 등 다양한 기업에서 채용 연계할 예정으로 기업마다 인재상 및 반영 요소가 다르기 때문에 중점 요소에 대해 안내드리기 어려운 점 양해 부탁드립니다. 채용에 대한 부분은 교육을 참여하시면 추후 관련 부서를 통해 안내드릴 예정입니다.
범주 : 50


In [106]:
question = '배가 너무 불러요'
ans, intent = get_answer(question)
print("=" * 120)
print("질문 :", question)
print("=" * 120)
print("대답 :", ans)
print("=" * 120)
print("범주 :", intent + 1)
print("=" * 120)

질문 : 배가 너무 불러요
대답 : 소화제 챙겨드세요.
범주 : 22


In [107]:
question = '4년제 졸업자만 지원 가능한가요?'
ans, intent = get_answer(question)
print("=" * 120)
print("질문 :", question)
print("=" * 120)
print("대답 :", ans)
print("=" * 120)
print("범주 :", intent + 1)
print("=" * 120)

질문 : 4년제 졸업자만 지원 가능한가요?
대답 : KT 에이블스쿨은 정규 4년제 대학 졸업자 및 졸업예정자 중 만 34세 이하 미취업자를 대상으로 하는 교육입니다.
단, 모집시점에 만 35세여도 해당연도 1월 1일 이후 생일자는 지원이 가능합니다.
전공은 상관 없습니다.
범주 : 31


In [108]:
question = '채용 연계가 가능한가요?'
ans, intent = get_answer(question)
print("=" * 120)
print("질문 :", question)
print("=" * 120)
print("대답 :", ans)
print("=" * 120)
print("범주 :", intent + 1)
print("=" * 120)

질문 : 채용 연계가 가능한가요?
대답 : KT 에이블스쿨은 미래인재육성 + 청년 IT/SW/DX 분야 일자리 창출을 위해서 KT 와 정부가 함께 하는 프로그램입니다. 에이블러 여러분들의 취업기회를 위해 Job Searching 세션, 취업컨설팅, 잡페어 등 여러 프로그램들을 운영하고 있습니다.
KT 직접채용은 그 일환 중 하나이며, KT에 합격한 1기 수료생의 경우 해당지역에 배치되었습니다. KT 외에 KT 그룹사, 주요 협력사, AI원팀 기업, 스타트업 등의 채용은 지역 구분 없이 진행됩니다. 채용 상황에 따라 변경될 수 있어 교육 수료 후 채용 연계 시 상세 안내 드리겠습니다. KT 그룹, AI 원팀 기업/스타트업 100 등 다양한 기업에서 채용 연계할 예정으로 기업마다 인재상 및 반영 요소가 다르기 때문에 중점 요소에 대해 안내드리기 어려운 점 양해 부탁드립니다. 채용에 대한 부분은 교육을 참여하시면 추후 관련 부서를 통해 안내드릴 예정입니다.
범주 : 50


In [109]:
question = '대면 교육시 교통비를 지원해주나요?'
ans, intent = get_answer(question)
print("=" * 120)
print("질문 :", question)
print("=" * 120)
print("대답 :", ans)
print("=" * 120)
print("범주 :", intent + 1)
print("=" * 120)

질문 : 대면 교육시 교통비를 지원해주나요?
대답 : 대중교통을 이용해 보세요.
범주 : 15


In [110]:
question = '노트북이 안돼'
ans, intent = get_answer(question)
print("=" * 120)
print("질문 :", question)
print("=" * 120)
print("대답 :", ans)
print("=" * 120)
print("범주 :", intent + 1)
print("=" * 120)

질문 : 노트북이 안돼
대답 : as 맡기세요.
범주 : 1


In [111]:
question = '놀고 싶다'
ans, intent = get_answer(question)
print("=" * 120)
print("질문 :", question)
print("=" * 120)
print("대답 :", ans)
print("=" * 120)
print("범주 :", intent + 1)
print("=" * 120)

질문 : 놀고 싶다
대답 : 저도요!!
범주 : 27


In [112]:
question = '나이제한이 있나요?'
ans, intent = get_answer(question)
print("=" * 120)
print("질문 :", question)
print("=" * 120)
print("대답 :", ans)
print("=" * 120)
print("범주 :", intent + 1)
print("=" * 120)

질문 : 나이제한이 있나요?
대답 : KT 에이블스쿨은 정규 4년제 대학 졸업자 및 졸업예정자 중 만 34세 이하 미취업자를 대상으로 하는 교육입니다.
단, 모집시점에 만 35세여도 해당연도 1월 1일 이후 생일자는 지원이 가능합니다.
또한 전공의 종류와는 무관 합니다.
범주 : 31
